In [22]:
import requests
import json
from time import sleep
import pandas as pd
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils')
sys.path.append('/home/ndsviriden/mmcc/calc_service/app/web/validation/common')

## API connect and get result

In [23]:
url = 'http://127.0.0.1:5000'
url_roadcast = url+ '/roadcast'
url_calc = url + '/calculation'

with open("/home/ndsviriden/data_metro.txt", "r") as file:
#with open("/mnt/HARD/MinMax94/data/json_test/1.txt", "r") as file:
    data = json.load(file)

for k, v in data['global_forecast'].items():
    data['global_forecast'][k]['cloudiness'] = int(data['global_forecast'][k]['cloudiness'])
    data['global_forecast'][k]['p_weather'] = int(data['global_forecast'][k]['p_weather'])

In [24]:
%%time
with requests.Session() as session:
    session.headers = {'Content-type': 'application/json', 'Host': ".api.mmcc.pkcup.ru", 
                       'cache-control': "no-cache", #'postman-token': '727b926f-c7b5-854e-06d7-bea9df150a12', 
                       'User-Agent':'test'}
    
    response_post = session.post(url_calc, params={"key":"MeAuthKeyMeSmart"}, json=data)
    sleep(2) ## What should I do with it
    job_id = json.loads(response_post.text)['job_id']
    response_get = session.get(url_roadcast, params={"job_id": job_id, "key": "MeAuthKeyMeSmart"}, timeout=5)
    print(response_get.url)
    z = response_get.content
z

http://127.0.0.1:5000/roadcast?job_id=5b17abee67b560001c3a60e2&key=MeAuthKeyMeSmart
CPU times: user 10.3 ms, sys: 0 ns, total: 10.3 ms
Wall time: 2.06 s


In [17]:
response_post.content

b'{\n  "error": null, \n  "job_id": "5b16765e67b5600011961b77"\n}\n'

In [18]:
response_get.content

b'{\n  "emergency_report": "\\u0421\\u0435\\u0433\\u043e\\u0434\\u043d\\u044f \\u043f\\u0435\\u0440\\u0435\\u0445\\u043e\\u0434 \\u0447\\u0435\\u0440\\u0435\\u0437 0 c 15:00 \\u0434\\u043e 15:20 \\u043e\\u0442 -0.0 \\u0434\\u043e -0.5\\n\\u0421\\u0435\\u0433\\u043e\\u0434\\u043d\\u044f \\u0433\\u043e\\u043b\\u043e\\u043b\\u0435\\u0434\\u0438\\u0446\\u0430 c 15:20 \\u0434\\u043e 16:20\\n\\u0421\\u0435\\u0433\\u043e\\u0434\\u043d\\u044f \\u0433\\u043e\\u043b\\u043e\\u043b\\u0435\\u0434 c 19:00 \\u0434\\u043e 20:00\\n\\u0421\\u0435\\u0433\\u043e\\u0434\\u043d\\u044f \\u043f\\u0435\\u0440\\u0435\\u0445\\u043e\\u0434 \\u0447\\u0435\\u0440\\u0435\\u0437 0 c 19:00 \\u0434\\u043e 20:00 \\u043e\\u0442 -1.0 \\u0434\\u043e 1.0\\n\\u0421\\u0435\\u0433\\u043e\\u0434\\u043d\\u044f \\u0434\\u043e\\u0436\\u0434\\u044c c 19:00 \\u0434\\u043e 21:00 \\u043e\\u0442 2.0 \\u0434\\u043e 3.0 \\u043c\\u043c/\\u0447\\n\\u0421\\u0435\\u0433\\u043e\\u0434\\u043d\\u044f \\u043f\\u043e\\u0445\\u043e\\u043b\\u043e\\